# About the notebook

The notebook is based on [GTC 2017 Numba Tutorial Notebook 4: Writing CUDA Kernels](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb). I have simplified some parts and added some information to clarify some concepts.

# The CUDA Programming Model

Ufuncs (and generalized ufuncs mentioned in the bonus notebook at the end of the tutorial) are the easiest way in Numba to use the GPU, and present an abstraction that requires minimal understanding of the CUDA programming model. However, not all functions can be written as ufuncs. Many problems require greater flexibility, in which case you want to write a CUDA kernel, the topic of this notebook.

Fully explaining the CUDA programming model is beyond the scope of this tutorial. We highly recommend that everyone writing CUDA kernels with Numba take the time to read Chapters 1 and 2 of the CUDA C Programming Guide:

**grid of thread block**

threadIdx is a 3-component vector, so that threads can be identified using a one-dimensional, two-dimensional, or three-dimensional thread index, forming a one-dimensional, two-dimensional, or three-dimensional block of threads, called a thread block

Each block within the grid can be identified by a one-dimensional, two-dimensional, or three-dimensional index accessible within the kernel through the built-in blockIdx variable. The dimension of the thread block is accessible within the kernel through the built-in blockDim variable.


**host memory and device memory**
Unified Memory provides managed memory to bridge the host and device memory spaces. Managed memory is accessible from all CPUs and GPUs in the system as a single, coherent memory image with a common address space. This capability enables oversubscription of device memory and can greatly simplify the task of porting applications by eliminating the need to explicitly mirror data on host and device. Se


**Compute Capability**

The compute capability of a device is represented by a version number, also sometimes called its "SM version". This version number identifies the features supported by the GPU hardware and is used by applications at runtime to determine which hardware features and/or instructions are available on the present GPU.

-----

We will be writing a kernel that decribes the execution of a single thread in this hierarchy. The CUDA compiler and driver will execute our kernel across a thread grid that is divided into blocks of threads. 
- **Threads within the same block can exchange data** very easily during the execution of a kernel, whereas 
- **threads in different blocks should generally not communicate with each other** (with a few exceptions).

Deciding the best size for the CUDA thread grid is a complex problem (and depends on both the algorithm and the specific GPU compute capability), but here are some very rough heuristics that we follow:

- the size of a block should be **a multiple of 32 threads**, with typical block sizes **between 128 and 512 threads per block**.
- the size of the grid should ensure the full GPU is utilized where possible. Launching a grid where the number of blocks is 2x-4x the number of "multiprocessors" on the GPU is a good starting place. Something in the range of 20 - 100 blocks is usually a good starting point.
- The CUDA kernel launch overhead does depend on the number of blocks, so we find it best not to launch a grid where the number of threads equals the number of input elements when the input size is very big. We'll show a pattern for dealing with large inputs below.

Each thread distinguishes itself from the other threads using its unique **thread (threadIdx) and block (blockIdx) index values**, which can be multidimensional if launched that way.

# A First Example

This all will be a little overwhelming at first, so let's start with a concrete example. Let's write our addition function for 1D NumPy arrays. CUDA kernels are compiled using the numba.cuda.jit decorator (not to be confused with the numba.jit decorator for the CPU):

In [3]:
from numba import cuda
import numpy as np

In [2]:
@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [4]:
# Prepare data
n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

In [5]:
# Specify memory allocation
threads_per_block = 128
blocks_per_grid = 30

In [6]:
add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid
    print(tx)
    
    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [16]:
n = 10
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

# Specify memory allocation
threads_per_block = 32
blocks_per_grid = 1

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out)

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


In [7]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [8]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

In [9]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)

1.05 ms ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); out_device.copy_to_host()

242 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
# CPU input/output arrays, implied synchronization for memory copies
%time add_kernel[blocks_per_grid, threads_per_block](x, y, out)

CPU times: user 194 µs, sys: 8.07 ms, total: 8.26 ms
Wall time: 4.34 ms


In [12]:
# GPU input/output arrays, no synchronization (but force sync before and after)
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)
cuda.synchronize()

CPU times: user 570 µs, sys: 184 µs, total: 754 µs
Wall time: 700 µs


In [13]:
# GPU input/output arrays, include explicit synchronization in timing
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); cuda.synchronize()

CPU times: user 1.69 ms, sys: 548 µs, total: 2.24 ms
Wall time: 1.4 ms


Always be sure to synchronize with the GPU when benchmarking CUDA kernels!

# Exercise